In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the HTML content from the URL
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
response = requests.get(url)
html_content = response.text

# Function to get the list of current S&P 500 companies
def get_current_companies(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', id='constituents')
    companies = []
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all('td')
        company = cols[0].text.strip()
        companies.append(company)
    return companies

# Get the list of current S&P 500 tickers
current_tickers = get_current_companies(html_content)

# Display the results
print("Current S&P 500 tickers:")
print('Number of companies:', len(current_tickers))
print(current_tickers)

Current S&P 500 tickers:
Number of companies: 503
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA

In [10]:
# Finde tickers that has non alphabetic characters
non_alpha_tickers = [ticker for ticker in current_tickers if not ticker.isalpha()]
print("Tickers with non-alphabetic characters:")
print(non_alpha_tickers)

# Fix the non-alphabetic tickers
for ticker in non_alpha_tickers:
    new_ticker = ticker.replace('.', '-')
    print(f"Replacing {ticker} with {new_ticker}")
    current_tickers = [new_ticker if x == ticker else x for x in current_tickers]

Tickers with non-alphabetic characters:
['BRK.B', 'BF.B']
Replacing BRK.B with BRK-B
Replacing BF.B with BF-B


In [2]:
# Function to get the selected changes
def get_selected_changes(html):
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table', class_='wikitable')
    changes = []
    for table in tables:
        # Check if the table is under the "Selected changes to the list of S&P 500 components" section
        if table.find_previous('h2').find('span', id='Selected_changes_to_the_list_of_S&P_500_components'):
            for row in table.find_all('tr')[1:]:  # Skip header row
                cols = row.find_all('td')
                if len(cols) >= 4:
                    date = cols[0].text.strip()
                    ticker_added = cols[1].text.strip()
                    ticker_removed = cols[3].text.strip()
                    changes.append({
                        'date': date,
                        'ticker_added': ticker_added,
                        'ticker_removed': ticker_removed
                    })
            break
    return changes

# Get the selected changes
selected_changes = get_selected_changes(html_content)

# Convert the selected changes to a DataFrame
selected_changes_df = pd.DataFrame(selected_changes)

# Convert date format from 'Month Day, Year' to 'dd/mm/yyyy'
selected_changes_df['date'] = pd.to_datetime(selected_changes_df['date']).dt.strftime('%d/%m/%Y')

# Display the DataFrame
print("\nSelected Changes DataFrame:")

# Convert to datetime, extract the start of the quarter, and format back to the desired format
selected_changes_df['quarter'] = pd.to_datetime(selected_changes_df['date'], format='%d/%m/%Y').dt.to_period('Q').dt.start_time.dt.strftime('%d/%m/%Y')
# Convert quarter to add tipe of 'Q1 2024', 'Q4 2023', 'Q3 2023', 'Q2 2023', 'Q1 2023'
selected_changes_df['quarter name'] = pd.to_datetime(selected_changes_df['date'], format='%d/%m/%Y').dt.to_period('Q').dt.strftime(' %Y Q%q')

selected_changes_df


Selected Changes DataFrame:


,date,ticker_added,ticker_removed,quarter,quarter name
0,08/05/2024,VST,PXD,01/04/2024,2024 Q2
1,03/04/2024,,XRAY,01/04/2024,2024 Q2
2,03/04/2024,,VFC,01/04/2024,2024 Q2
3,02/04/2024,GEV,,01/04/2024,2024 Q2
4,01/04/2024,SOLV,,01/04/2024,2024 Q2
...,...,...,...,...,...
340,09/06/1999,WLP,HPH,01/04/1999,1999 Q2
341,11/12/1998,FSR,LDW,01/10/1998,1998 Q4
342,11/12/1998,CCL,GRN,01/10/1998,1998 Q4
343,11/12/1998,CPWR,SUN,01/10/1998,1998 Q4


In [3]:
# Test multiculiniarity between the indicators
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Function to calculate the VIF for each variable
def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif_data

def drop_high_vif(data):
    data = data.replace([np.inf, -np.inf], np.nan).dropna()
    count = 0
    total_columns = len(data.columns)
    data = data.copy()  # Work on a copy of the data
    while True:
        vif = calculate_vif(data)
        max_vif = vif['VIF'].max()
        if max_vif > 10:
            max_vif_column = vif.loc[vif['VIF'].idxmax(), 'feature']
            print(f"Dropping column '{max_vif_column}' with VIF of {max_vif}")
            data.drop(columns=max_vif_column, inplace=True)
            count += 1
        else:
            break
    print(f"\nTotal columns dropped: {count} out of {total_columns}")
    return data

In [6]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime

def fetch_quarterly_indicators(start_date='2000-01-01', end_date=datetime.today().strftime('%Y-%m-%d')):
    # Define the indicators and their corresponding FRED codes
    indicators = {
        'GDP': 'GDP',
        'Unemployment Rate': 'UNRATE',
        'CPI': 'CPIAUCSL',
        'PPI': 'PPIACO',
        'Federal Funds Rate': 'FEDFUNDS',
        '10-Year Treasury Yield': 'GS10',
        'Consumer Confidence Index': 'UMCSENT',
        'Retail Sales': 'RSAFS',
        'Housing Starts': 'HOUST',
        'Durable Goods Orders': 'DGORDER',
        'Industrial Production': 'INDPRO',
        'Trade Balance': 'BOPGSTB',
        'Personal Income': 'PI',
        'Business Inventories': 'BUSINV'
    }
    
    # Create an empty DataFrame to store the results
    data = pd.DataFrame()
    
    # Fetch data for each indicator
    for indicator, fred_code in indicators.items():
        df = web.DataReader(fred_code, 'fred', start_date, end_date)
        df = df.resample('Q').mean()  # Resample to quarterly frequency and take the mean
        df.rename(columns={fred_code: indicator}, inplace=True)
        data = pd.concat([data, df], axis=1)
    
    return data

# Fetch quarterly data since 2000
economical_indicators = fetch_quarterly_indicators()

# Add quarter index in the format 'Q1 2024'
economical_indicators['quarter'] = economical_indicators.index.to_period('Q').strftime('%Y Q%q')
# Set the quarter as the index
economical_indicators.set_index('quarter', inplace=True)

# Calculate quarter-over-quarter changes
economical_indicators = economical_indicators.pct_change() * 100  # Convert to percentage

In [7]:
# Drop columns with high VIF values
economical_indicators = drop_high_vif(economical_indicators)

# Display the results
print("Economical Indicators DataFrame:")
economical_indicators.head()

Dropping column 'Industrial Production' with VIF of 11.973764726642306

Total columns dropped: 1 out of 14
Economical Indicators DataFrame:


,GDP,Unemployment Rate,CPI,PPI,Federal Funds Rate,10-Year Treasury Yield,Consumer Confidence Index,Retail Sales,Housing Starts,Durable Goods Orders,Trade Balance,Personal Income,Business Inventories
quarter,,,,,,,,,,,,,
2000 Q2,2.454875,-2.479339,0.783853,1.851376,10.510863,-4.681070,-1.240920,0.074332,-4.360056,4.501897,0.784161,1.680865,1.706805
2000 Q3,0.687421,1.694915,0.913864,1.312800,3.931987,-4.587156,-1.195219,0.886670,-5.231092,-4.218298,7.199891,1.895193,1.324908
2000 Q4,1.139534,-2.500000,0.712909,1.320708,-0.715746,-5.542986,-3.287841,0.686554,2.682332,-2.376190,5.295400,1.189201,1.150361
2001 Q1,0.330470,8.547009,0.956572,1.647811,-13.594233,-9.281437,-11.225144,0.640726,3.950777,-4.372139,-2.216618,1.815397,-0.202046
2001 Q2,1.229858,3.937008,0.701156,-1.112993,-22.646007,4.356436,-1.372832,1.036860,1.557632,-2.519178,-10.839028,0.412239,-1.003856
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023 Q2,0.930166,1.904762,0.751542,-1.484616,10.479705,-1.462523,-3.560372,-0.116640,6.306306,3.951902,1.000255,0.989966,-0.271092
2023 Q3,2.021638,3.738318,0.846112,0.786329,5.410822,15.491651,11.663991,1.470399,-5.153459,-0.792028,-7.968348,0.951838,0.195822
2023 Q4,1.256314,0.900901,0.674652,-1.626973,1.330798,6.987952,-6.660278,0.803480,7.292924,0.583508,3.326751,0.700867,0.242440


In [5]:
import pandas as pd
from fredapi import Fred

# Replace 'your_api_key' with your actual FRED API key
fred = Fred(api_key='6e1efd417db3c8b5678331eb083b3d3a')

# Function to search for indicators
def search_indicators(keyword, limit=100):
    search_results = fred.search(keyword, order_by='popularity', limit=limit)
    return search_results

# Search for indicators related to 'social'
keyword = 'social'
indicators = search_indicators(keyword)

all_monthly_indicators = indicators[indicators['frequency'] == 'Monthly'][['title', 'id']]
# Use all monthly indicators
indicators_dict = all_monthly_indicators.set_index('title').to_dict()['id']

# Dictionary to store dataframes for each indicator
data_frames = {}

# Function to fetch data for a given indicator
def fetch_data(series_id):
    # If the series ID is not found, continue to the next indicator
    try:
        data = fred.get_series(series_id)
    except:
        return None
    data = data.resample('Q').mean()  # Resample to quarterly data
    return data

# Fetch data for each indicator and store in the dictionary
for indicator, series_id in indicators_dict.items():
    data = fetch_data(series_id)
    if data is not None:
        data_frames[indicator] = data

# Combine dataframes into a single dataframe
combined_df = pd.concat(data_frames, axis=1)

# Calculate quarter-to-quarter change
social_indicators = combined_df.pct_change() * 100  # Convert to percentage
# social_indicators = combined_df.diff().dropna() 

# Slice the dataframe to only include data from 2000 onwards
social_indicators = social_indicators['2000':]

social_indicators['quarter'] = social_indicators.index.to_period('Q').strftime('%Y Q%q')
social_indicators.set_index('quarter', inplace=True)

In [153]:
# Drop columns with high VIF values
social_indicators = drop_high_vif(social_indicators)

print("Social Indicators DataFrame:")
social_indicators.head()

Dropping column 'Indexes of Aggregate Weekly Payrolls of Production and Nonsupervisory Employees, Total Private' with VIF of 27733.941217416217
Dropping column 'Average Weekly Earnings of Production and Nonsupervisory Employees, Manufacturing' with VIF of 397.3455702958243
Dropping column 'Average Weekly Earnings of Production and Nonsupervisory Employees, Total Private' with VIF of 268.26128186526233
Dropping column 'Average Hourly Earnings of Production and Nonsupervisory Employees, Goods-Producing' with VIF of 123.5553144586158
Dropping column 'Personal current transfer receipts: Government social benefits to persons' with VIF of 95.79181091547315
Dropping column 'Average Hourly Earnings of Production and Nonsupervisory Employees, Manufacturing' with VIF of 50.250329234750126
Dropping column 'All Employees, Health Care and Social Assistance' with VIF of 39.217110203687895
Dropping column 'Production and Nonsupervisory Employees, Total Private' with VIF of 28.278422915345445
Dropping

,Personal Income,"Average Weekly Hours of Production and Nonsupervisory Employees, Manufacturing",Household Estimates,Business Applications: Retail Trade in the United States,Business Applications: Total for All NAICS in the United States,"Average Weekly Hours of Production and Nonsupervisory Employees, Total Private",Personal current transfer receipts: Government social benefits to persons: Social security,"Average Hourly Earnings of Production and Nonsupervisory Employees, Durable Goods",Job Openings: Health Care and Social Assistance,"Average Hourly Earnings of Production and Nonsupervisory Employees, Construction",...,Quits: Health Care and Social Assistance,"Average Hourly Earnings of Production and Nonsupervisory Employees, Leisure and Hospitality",Personal current transfer receipts: Government social benefits to persons: Medicaid,Harmonized Index of Consumer Prices: Actual Rentals for Housing for Euro Area (18 countries),Business Applications: Transportation and Warehousing in the United States,"Average Hourly Earnings of Production and Nonsupervisory Employees, Retail Trade","Average Hourly Earnings of Production and Nonsupervisory Employees, Transportation and Warehousing",M1 for Turkey,Hires: Health Care and Social Assistance,Layoffs and Discharges: Health Care and Social Assistance
quarter,,,,,,,,,,,,,,,,,,,,,
2004 Q4,2.084264,-0.490597,0.632232,9.592754,14.836085,0.197824,1.076448,1.485443,0.845921,0.294832,...,-0.268817,1.055697,2.162350,0.548141,5.168964,0.826446,0.344269,3.604737,1.114650,-22.506394
2005 Q1,0.084274,0.082169,0.194956,0.061821,16.637336,-0.098717,3.072853,0.468384,7.130018,0.121044,...,6.064690,0.396254,2.331636,0.466145,5.032512,0.901639,0.161453,0.370556,5.354331,24.422442
2005 Q2,1.608286,-0.492611,0.087544,1.284014,-2.991537,0.000000,1.256992,0.407925,4.753915,0.466321,...,3.684879,0.574094,3.130185,0.460050,3.945072,0.406174,0.120895,13.419358,2.167414,3.183024
2005 Q3,1.566255,0.330033,0.539640,-5.019228,-14.926648,0.000000,0.233979,0.696460,1.494928,0.567303,...,-1.225490,0.749197,-3.432003,0.532310,-1.051798,0.080906,0.804991,9.869985,-0.950988,-4.627249
2005 Q4,1.789390,0.904605,0.420849,14.751832,11.440775,0.197628,0.667877,1.229587,-7.101526,0.358974,...,-2.233251,0.566572,1.754776,0.408799,12.192233,0.000000,0.539030,30.587590,-2.880355,-18.598383


In [41]:
import yfinance as yf
import pandas as pd
import warnings
from tqdm import tqdm

def get_qoq_metrics(current_tickers):
    # Suppress specific warnings
    warnings.filterwarnings("ignore", category=UserWarning, message=".*Converting to PeriodArray/Index representation will drop timezone information.*")

    # Define a list to store the results as DataFrames
    results = []

    # Loop through each ticker with a progress bar
    for ticker in tqdm(current_tickers, desc="Processing tickers"):
        try:
            # Fetch historical market data since 2000
            stock = yf.Ticker(ticker)
            hist = stock.history(start="2000-01-01", interval="1d")

            # Calculate daily volume change and daily rate change
            hist['Daily_Volume_Change'] = hist['Volume'].pct_change() * 100
            hist['Daily_Rate_Change'] = hist['Close'].pct_change() * 100

            # Calculate daily returns
            hist['Daily_Return'] = hist['Close'].pct_change() * 100

            # Calculate ROC for each quarter
            hist['Quarter'] = hist.index.to_period('Q').strftime('%Y Q%q')
            grouped = hist.groupby('Quarter').apply(calculate_roc).reset_index()
            grouped.columns = ['Quarter', 'Volume_ROC']

            # Resample data to quarterly frequency
            quarterly_data = hist.resample('Q').agg({
                'Close': 'last',
                'Volume': 'sum',
                'Daily_Volume_Change': 'mean',
                'High': 'max',
                'Low': 'min',
                'Daily_Return': 'mean'
            })

            # Calculate QoQ change rates for Close and Volume
            quarterly_data['QoQ_Close_Change'] = quarterly_data['Close'].pct_change() * 100
            quarterly_data['QoQ_Volume_Change'] = quarterly_data['Volume'].pct_change() * 100

            # Calculate quarterly volatility (standard deviation of daily returns)
            quarterly_data['Quarterly_Volatility'] = hist['Daily_Return'].resample('Q').std()

            # Calculate quarterly high-low spread
            quarterly_data['Quarterly_High_Low_Spread'] = quarterly_data['High'] - quarterly_data['Low']

            # Format the index to be in the format of %Y Q%q
            quarterly_data.index = quarterly_data.index.to_period('Q').strftime('%Y Q%q')

            # Add a ticker column
            quarterly_data['Ticker'] = ticker

            # Change index name to 'Quarter'
            quarterly_data.index.names = ['Quarter']

            # Merge the ROC data with the quarterly data
            quarterly_data = quarterly_data.reset_index().rename(columns={'index': 'Quarter'})
            quarterly_data = quarterly_data.merge(grouped, on='Quarter', how='left')

            # Append the DataFrame to the results list
            results.append(quarterly_data)

        except Exception as e:
            print(f"Error processing ticker {ticker}: {e}")
            continue

    # Combine all results into a single DataFrame
    combined_results = pd.concat(results)
    
    # Exclude columns Close and Volume
    combined_results.drop(columns=['Close', 'Volume', 'High', 'Low', 'Daily_Return'], inplace=True)

    # Set the index to be a MultiIndex with 'Quarter' and 'Ticker'
    combined_results.set_index(['Quarter', 'Ticker'], inplace=True)

    return combined_results

def calculate_roc(df):
    n = len(df)
    mid_point = n // 2
    early_part = df.iloc[:mid_point]
    late_part = df.iloc[mid_point:]

    avg_early_volume_change = early_part['Daily_Volume_Change'].mean()
    avg_late_volume_change = late_part['Daily_Volume_Change'].mean()

    if avg_early_volume_change != 0:
        roc = (avg_late_volume_change - avg_early_volume_change) / avg_early_volume_change * 100
    else:
        roc = float('nan')
    return roc

# Fetch quarterly metrics for the S&P 500 companies
qoq_metrics = get_qoq_metrics(current_tickers)
qoq_metrics

Processing tickers: 100%|██████████| 3/3 [00:01<00:00,  2.09it/s]


,,Daily_Volume_Change,QoQ_Close_Change,QoQ_Volume_Change,Quarterly_Volatility,Quarterly_High_Low_Spread,Volume_ROC
Quarter,Ticker,,,,,,
2000 Q1,AAPL,10.195773,NaN,NaN,4.772212,0.482173,-17.828916
2000 Q2,AAPL,11.332785,-22.871653,-6.992387,4.411406,0.446316,6.918140
2000 Q3,AAPL,24.720753,-50.835370,-2.542102,7.705252,0.585023,171.512065
2000 Q4,AAPL,4.618879,-42.232979,51.557743,4.423293,0.198153,-496.621783
2001 Q1,AAPL,7.582765,48.369781,-19.669368,4.438993,0.140594,-0.321256
...,...,...,...,...,...,...,...
2023 Q2,GOOGL,3.856719,15.395732,-16.254992,1.673605,27.109993,-63.780811
2023 Q3,GOOGL,5.206795,9.323312,-14.520735,1.610628,23.810005,-0.576237
2023 Q4,GOOGL,4.887096,6.747671,3.184823,1.843449,22.469994,236.099761


In [42]:
# Replace infinite and NaN values with zeros
qoq_metrics = qoq_metrics.replace([np.inf, -np.inf, np.nan], 0)

# Drop Daily_Rate_Change
# qoq_metrics.drop(columns=['Daily_Rate_Change'], inplace=True)

# Print the VIF values for the QoQ metrics
print("VIF values for the QoQ metrics:")
calculate_vif(qoq_metrics)

VIF values for the QoQ metrics:


,feature,VIF
0,Daily_Volume_Change,4.258602
1,QoQ_Close_Change,1.155722
2,QoQ_Volume_Change,1.052268
3,Quarterly_Volatility,4.680313
4,Quarterly_High_Low_Spread,1.413805
5,Volume_ROC,1.031760


In [43]:
def get_financials_table(ticker):
    ticker = ticker.lower()
    # URLs for the different financial sections
    urls = {
        "Income": f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly",
        "Balance Sheet": f"https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/?p=quarterly",
        "Cash Flow": f"https://stockanalysis.com/stocks/{ticker}/financials/cash-flow-statement/?p=quarterly",
        "Ratios": f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly",
    }

    # Initialize an empty DataFrame to concatenate all the data
    concatenated_df = pd.DataFrame()

    for section_name, url in urls.items():
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        section = soup.find('table', {'data-test': 'financials'})
        if section is None:
            print(f"No data found for {ticker} in {section_name} section.")
            continue
        headers = [th.get_text() for th in section.find('thead').find_all('th')]
        rows = section.find('tbody').find_all('tr')
        data = []
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append(cols)
        df = pd.DataFrame(data, columns=headers)
        concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)
    
    # Remove column with the word Quarters and Current
    concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.str.contains('Quarters|Current')]
    if 'Quarter Ended' in concatenated_df.columns:
        df_columns = concatenated_df['Quarter Ended'].to_list()
        concatenated_df = concatenated_df.drop(columns='Quarter Ended')
        concatenated_df = concatenated_df.T
        concatenated_df.columns = df_columns
        # Adjust for duplicate index values caused by multiple financial sections
        concatenated_df = concatenated_df.reset_index().rename(columns={'index': 'Date'})
    
    return concatenated_df

def concatenate_financial_tables(stock_list):
    all_columns = set()  # Set to store all unique column names across tables
    table_data = []  # List to store processed table data
    countr = 0
    for symbol in stock_list:
        countr += 1
        print(f"Processing {symbol}...{countr}")
       
        try:
            table = get_financials_table(symbol)  # Fetch the financial table for the current symbol
            table.reset_index(inplace=True, drop=True)
            table['symbol'] = symbol  # Add 'symbol' column
            
            # Ensure table has unique column names
            table = table.T.drop_duplicates().T
            
            all_columns.update(table.columns)  # Update the set with new columns from this table
            table_data.append(table)  # Add processed table data to the list
            
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
    
    concatenated_tables = pd.DataFrame(columns=list(all_columns))  # Initialize with unique columns
    
    for table in table_data:
            # Ensure table has unique columns before reindexing
            table = table.loc[:, ~table.columns.duplicated()]
            # Reindex each table to match the full set of columns, filling missing ones with NaN
            table_reindexed = table.reindex(columns=concatenated_tables.columns)
            concatenated_tables = pd.concat([concatenated_tables, table_reindexed], axis=0, ignore_index=True)

    # sort columns
    concatenated_tables = concatenated_tables.reindex(sorted(concatenated_tables.columns), axis=1)

    # reorder the date and symbol columns
    concatenated_tables = concatenated_tables[['Date', 'symbol'] + [col for col in concatenated_tables.columns if col not in ['Date', 'symbol']]]

    return concatenated_tables

# Assuming stock_lst is defined
concatenated_financials = concatenate_financial_tables(stock_list = test)

# Save the concatenated table to a CSV file
# concatenated_financials.to_csv('financials_concatenated.csv', index=False)

Processing AAPL...1
Processing MSFT...2
Processing GOOGL...3


In [67]:
# Load the concatenated table from a CSV file
path = r"C:\Users\naama\Desktop\Chen\לימודים\מדע המידע למימון\InvestingStrategy\financials_concatenated.csv"
financial_indicators = pd.read_csv(path)

C:\Users\naama\AppData\Local\Temp\ipykernel_36688\3933112158.py:3: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  financial_indicators = pd.read_csv(path)


In [68]:
# Drop columns that contain more than 40% NaN values and print the columns that were dropped
dropped_columns = financial_indicators.columns[financial_indicators.isnull().mean() > 0.4]
financial_indicators.drop(columns=dropped_columns, inplace=True)
print("Dropped columns:", '\n')
for col in dropped_columns:
    print(col)

Dropped columns: 

Asset Turnover
Cash & Cash Equivalents
Common Stock Issued
Deferred Revenue
Earnings Yield
FCF Yield
Goodwill
Intangible Assets
Interest Expense
Interest Income
Long-Term Investments
Net Cash / Debt Growth
Net Income Common
Preferred Dividends
Research & Development
Return on Assets (ROA)
Return on Equity (ROE)
Share Repurchases
Short-Term Investments
Total Liabilities and Equity


In [69]:
# Chagne Date to Quarter in the format 'Q1 2024'
financial_indicators['Date'] = pd.to_datetime(financial_indicators['Date']).dt.to_period('Q').astype(str).str.replace('Q', ' Q')

# Set the 'Date' and 'symbol' columns as the index
financial_indicators.set_index(['Date', 'symbol'], inplace=True)

# Change Date to Quarter
financial_indicators.index.names = ['Quarter', 'symbol']

# Remove "%" and ',' from the data
financial_indicators = financial_indicators.replace({'%': '', ',': ''}, regex=True)

# Change all columns to numeric
financial_indicators = financial_indicators.apply(pd.to_numeric, errors='coerce')

# Drop columns with high VIF values
financial_indicators = drop_high_vif(financial_indicators)

# Display the results
print("Financial Indicators DataFrame:")
financial_indicators

c:\Users\naama\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping column 'Buyback Yield / Dilution' with VIF of inf
Dropping column 'Market Capitalization' with VIF of 1064303350436.1328
Dropping column 'Total Assets' with VIF of 78328239586.24431
Dropping column 'Total Liabilities' with VIF of 31035763402.732384
Dropping column 'Total Current Assets' with VIF of 20607054459.213467
Dropping column 'Investing Cash Flow' with VIF of 14248132621.191856
Dropping column 'Shareholders' Equity' with VIF of 12826145640.874432
Dropping column 'Total Long-Term Liabilities' with VIF of 10808072762.676426
Dropping column 'Total Debt' with VIF of 8400286924.717106
Dropping column 'Net Income' with VIF of 7961243166.3082485
Dropping column 'Revenue' with VIF of 4612181920.22745
Dropping column 'EBIT' with VIF of 3763103622.702369
Dropping column 'Total Current Liabilities' with VIF of 3615538425.0301123
Dropping column 'Operating Income' with VIF of 3372852509.8646326
Dropping column 'Pretax Income' with VIF of 3170394741.752824
Dropping column 'EBITDA' w

,,Accounts Payable,Acquisitions,Book Value Per Share,Capital Expenditures,Cash & Equivalents,Cash Growth,Change in Investments,Common Stock,Comprehensive Income,Current Debt,...,PB Ratio,PE Ratio,Payout Ratio,Profit Margin,Quick Ratio,Return on Capital (ROIC),Revenue Growth (YoY),"Selling, General & Admin",Share Issuance / Repurchase,Shares Change
Quarter,symbol,,,,,,,,,,,,,,,,,,,,,
2022 Q3,MMM,3063.0,478.0,24.75,-426.0,3404.0,-37.40,90.0,-26180.0,-7985.0,2097.0,...,4.47,9.57,21.9,44.77,0.87,14.02,-3.61,1998.0,-72.0,-2.78
2022 Q2,MMM,3273.0,0.0,24.07,-384.0,2722.0,-45.75,-154.0,-24165.0,-7362.0,2513.0,...,5.36,17.75,1064.3,0.90,0.80,17.62,-2.77,3023.0,63.0,-2.70
2021 Q3,MMM,2862.0,0.0,24.95,-342.0,4878.0,25.70,-45.0,-23281.0,-7620.0,2234.0,...,7.02,16.95,59.9,16.04,1.11,18.53,7.09,1819.0,-441.0,0.67
2021 Q2,MMM,2931.0,0.0,24.87,-383.0,4695.0,23.15,-292.0,-22890.0,-7486.0,2249.0,...,7.97,19.25,56.5,17.03,1.11,18.42,24.72,1746.0,-316.0,1.34
2021 Q1,MMM,2670.0,0.0,23.70,-278.0,4636.0,14.74,-110.0,-22728.0,-7767.0,1635.0,...,8.11,19.36,52.9,18.35,1.19,17.68,9.61,1808.0,62.0,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021 Q4,ZTS,436.0,0.0,9.61,-166.0,3485.0,-3.30,25.0,-1879.0,-764.0,0.0,...,25.41,56.68,28.7,21.05,2.57,18.97,8.85,593.0,-198.0,-0.67
2020 Q4,ZTS,457.0,-5.0,7.93,-152.0,3604.0,86.35,-19.0,-1160.0,-730.0,604.0,...,20.87,48.02,26.7,19.87,2.13,16.15,7.95,520.0,0.0,-0.31
2020 Q2,ZTS,335.0,-72.0,6.27,-87.0,3353.0,88.37,-4.0,-1210.0,-856.0,501.0,...,21.83,40.25,25.3,24.35,2.39,19.16,0.06,393.0,0.0,-0.87
